In [125]:
#hide
import core_constants as cc
import functions as fx
import requests
from bs4 import BeautifulSoup
import lxml
import time
import json
import pandas as pd
import sqlite3 as sql
import recordlinkage
import csv

In [179]:
url = 'https://en.wikipedia.org/wiki/2020_All-Big_12_Conference_football_team'
r = requests.get(url) 
contents = (r.text).replace("\n", "") 
wikiSoup = BeautifulSoup(contents, 'lxml')


In [180]:
firsth3 = wikiSoup.h3
players = []

In [188]:
targetedWiki = firsth3.find_all_next(string = True)
quitKeyword = 'Key'
removeKeywords = ['[', 'edit', ']']
positionKeywords = ['Quaterbacks', 'Running backs', 'Fullbacks', 'Wide receivers', 'Receivers', 'Centers', 'Guards', 'Tackles', 'Tight ends', 'Defensive linemen', 'Defensive ends', 'Defensive tackles', 'Linebackers', 'Defensive backs', 'Cornerbacks', 'Safeties', 'Kickers', 'Punter', 'Punters', 'All purpose/return specialist', 'All-purpose / Return specialists','Return specialist']
position = 'Quaterbacks'
playerName = ''
players = []
player = {}
for row in targetedWiki:
    row = row.lstrip()
    row = row.replace('†', '').replace('*', '').replace('#', '')
    #print (row)
    if (', ' != row[0:2] and '(' in row):
        try:
            playerName = row.split(',', 1)[0]
            college = (row.split(',', 1)[1]).split('(', 1)[0].lstrip().rstrip()
            allConfTeam = [int(char) for char in row if char.isdigit()]
            player['playerName'] = playerName
            player['position'] = position
            player['college'] = college
            player['allConferenceTeam'] = min(allConfTeam)
            players.append(player)
            player = {}
        except:
            print('ERROR: ' + row)
    elif (row == quitKeyword):
        break
    elif (row in positionKeywords):
        position = row
    elif ('(' not in row):
        playerName = row
    elif (row not in removeKeywords):
        try:
            college = row.split('(', 1)[0].replace(', ', '').rstrip()
            allConfTeam = [int(char) for char in row if char.isdigit()]
            player['playerName'] = playerName
            player['position'] = position
            player['college'] = college
            player['allConferenceTeam'] = min(allConfTeam)
            players.append(player)
            player = {}
        except:
            print('ERROR: ' + row)
    else:
        print('ERROR: ' + row)

players

[{'playerName': 'Sam Ehlinger',
  'position': 'Quaterbacks',
  'college': 'Texas',
  'allConferenceTeam': 2},
 {'playerName': 'Brock Purdy',
  'position': 'Quaterbacks',
  'college': 'Iowa State',
  'allConferenceTeam': 1},
 {'playerName': 'Spencer Rattler',
  'position': 'Quaterbacks',
  'college': 'Oklahoma',
  'allConferenceTeam': 1},
 {'playerName': 'Leddie Brown',
  'position': 'Running backs',
  'college': 'West Virginia',
  'allConferenceTeam': 1},
 {'playerName': 'Breece Hall',
  'position': 'Running backs',
  'college': 'Iowa State',
  'allConferenceTeam': 1},
 {'playerName': 'Chuba Hubbard',
  'position': 'Running backs',
  'college': 'Oklahoma State',
  'allConferenceTeam': 2},
 {'playerName': 'Jeremiah Hall',
  'position': 'Fullbacks',
  'college': 'Oklahoma',
  'allConferenceTeam': 1},
 {'playerName': 'Briley Moore',
  'position': 'Fullbacks',
  'college': 'Kansas State',
  'allConferenceTeam': 2},
 {'playerName': 'Creed Humphrey',
  'position': 'Centers',
  'college': 'Ok

In [201]:
inputDirectory = '..//scrapedData//'
dataset = 'allConf'

## Load the id config
idConfig = json.loads(open('..//config//idConfig.json', "r").read())

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourceFiles.json', "r").read())

#allConfData = mergeSourceFiles(dataset, inputDirectory, sourceFiles)
allConfData = fx.mergeSourceFiles('allConf_nu', inputDirectory, sourceFiles)

len(allConfData)

16618

In [202]:
for i in allConfData:
    if (i['College'] == 'wyoming'):
        print(i)
    if (i['College'] == 'maryland'):
        print(i)

{'Year': '2002', 'POSITION': 'RB', 'PlayerName': 'Chris Downs', 'College': 'maryland', 'Position': 'RB', 'AllConferenceTeam': '1'}
{'Year': '2002', 'POSITION': 'T', 'PlayerName': 'Matt Crawford', 'College': 'maryland', 'Position': 'T', 'AllConferenceTeam': '1'}
{'Year': '2002', 'POSITION': 'C', 'PlayerName': 'Todd Wike', 'College': 'maryland', 'Position': 'C', 'AllConferenceTeam': '1'}
{'Year': '2002', 'POSITION': 'PK', 'PlayerName': 'Nick Novak', 'College': 'maryland', 'Position': 'PK', 'AllConferenceTeam': '1'}
{'Year': '2002', 'POSITION': 'LB', 'PlayerName': 'E.J. Henderson', 'College': 'maryland', 'Position': 'LB', 'AllConferenceTeam': '1'}
{'Year': '2002', 'POSITION': 'DB', 'PlayerName': 'Domonique Foxworth', 'College': 'maryland', 'Position': 'DB', 'AllConferenceTeam': '1'}
{'Year': '2002', 'POSITION': 'SP', 'PlayerName': 'Steve Suter', 'College': 'maryland', 'Position': 'SP', 'AllConferenceTeam': '1'}
{'Year': '2002', 'POSITION': 'P', 'PlayerName': 'Brooks Barnard', 'College': '